
 <center>

<img src=https://www.cct.ie/wp-content/uploads/CCT_Logo_New_Aug_17-2.jpg width=600>

 </center>

<center>
 
<div style="text-align: center"> <span style="font-size: 20px;"> <b>Continuous Assessment 2  </b> </span></div>

<div style="text-align: center"> <span style="font-size:20px;"><b>Machine Learnig for Business</b> </span></div>
    
<div style="text-align: center"> <span style="font-size:16px;"><b>Muhammad Iqbal  </b> </span></div>
    
    
<b></b> 

<div style="text-align: center"> <span style="font-size: 14px;"><b> 2022455 - Ana Isabel Nieves Barcenas</b> </span></div>

### Table of Contents
<a id="Table of Contents"></a>

#### <a href="#Introduction">Introduction</a>

#### <a href="#Data Dictionary ">Data Dictionary</a>

#### <a href="#Characterisation of the data set">Characterisation of the data set</a>

#### <a href="#Data Preparation">Data Preparation</a>

* <a href="#Descrive stats">Descrive stats</a> 


* <a href="#Duplicates">Duplicates</a> 


* <a href="#Missing values">Missing values</a> 


#### <a href="#Conclusion">Conclusion</a>

#### <a href="#References">References</a>



# Introduction 

## Data Dictionary 

* Date. Categorical variable that tells us the date of the transactions (YYYY-MM-DD format). The column includes dates from 30/10/2016 to 09/04/2017.

* Time. Categorical variable that tells us the time of the transactions (HH:MM:SS format).

* Transaction. Quantitative variable that allows us to differentiate the transactions. The rows that share the same value in this field belong to the same transaction, that’s why the data set has less transactions than observations.

* Item. Categorical variable containing the products.

## Characterisation of the Data set 

In [41]:
#Data manipulation libraries
import pandas as pd
import numpy as np

#Visualizations
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

#for preprocessing
from mlxtend.preprocessing import TransactionEncoder


#for market basket analysis (using apriori)
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#for market basket analysis (using FP Growth)
from mlxtend.frequent_patterns import fpgrowth

# We can suppress the warnings
import warnings
warnings.filterwarnings('ignore') 


In [7]:
# Using Pandas to Import the  Accidentes dataset
df_bakery=pd.read_csv("BreadBasket_DMS.csv")


In [8]:
df_bakery.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam


In [9]:
df_bakery.shape

(21293, 4)

In [10]:
df_bakery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21293 entries, 0 to 21292
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         21293 non-null  object
 1   Time         21293 non-null  object
 2   Transaction  21293 non-null  int64 
 3   Item         21293 non-null  object
dtypes: int64(1), object(3)
memory usage: 665.5+ KB


In [35]:
# Count of unique customers
df_bakery['Transaction'].nunique()

9531

## EDA

In [11]:
df_bakery.describe()

,Transaction
count,21293.000000
mean,4951.990889
std,2787.758400
min,1.000000
25%,2548.000000
50%,5067.000000
75%,7329.000000
max,9684.000000


In [15]:
df_bakery.describe(include = object)

,Date,Time,Item
count,21293,21293,21293
unique,159,8240,95
top,2017-02-04,12:07:39,Coffee
freq,302,16,5471


## Feacture Engineering

In [36]:
df_bakery.isna().sum()

Date           0
Time           0
Transaction    0
Item           0
dtype: int64

In [37]:
print(df_bakery[df_bakery['Item']=='NONE'])

             Date      Time  Transaction  Item
26     2016-10-30  10:27:21           11  NONE
38     2016-10-30  10:34:36           15  NONE
39     2016-10-30  10:34:36           15  NONE
66     2016-10-30  11:05:30           29  NONE
80     2016-10-30  11:37:10           37  NONE
...           ...       ...          ...   ...
21108  2017-04-08  11:54:22         9590  NONE
21122  2017-04-08  12:58:25         9599  NONE
21254  2017-04-09  12:01:07         9666  NONE
21255  2017-04-09  12:04:13         9667  NONE
21266  2017-04-09  12:31:28         9672  NONE

[786 rows x 4 columns]


In [38]:
df_bakery.drop(df_bakery[df_bakery['Item']=='NONE'].index, inplace=True)

In [40]:
df_bakery.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20507 entries, 0 to 21292
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         20507 non-null  object
 1   Time         20507 non-null  object
 2   Transaction  20507 non-null  int64 
 3   Item         20507 non-null  object
dtypes: int64(1), object(3)
memory usage: 801.1+ KB


# Apriori 

In [42]:
transactions_str = df.groupby(['Transaction', 'Item'])['Item'].count().reset_index(name ='Count')
transactions_str

,Transaction,Item,Count
0,1,Bread,1
1,2,Scandinavian,1
2,3,Cookies,1
3,3,Hot chocolate,1
4,3,Jam,1
...,...,...,...
19635,9682,Tacos/Fajita,1
19636,9682,Tea,1
19637,9683,Coffee,1
19638,9683,Pastry,1


In [43]:
# making a mxn matrice where m=transaction and n=items and each row represents whether the item was in the transaction or not
my_basket = transactions_str.pivot_table(index='Transaction', columns='Item', values='Count', aggfunc='sum').fillna(0)


my_basket.head()

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# making a function which returns 0 or 1
# 0 means item was not in that transaction, 1 means item present in that transaction

def encode(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

# applying the function to the dataset

my_basket_sets = my_basket.applymap(encode)
my_basket_sets.head()

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
# using the 'apriori algorithm' with min_support=0.01 (1% of 9465)
# It means the item should be present in atleast 94 transaction out of 9465 transactions only when we considered that item in
# frequent itemset
frequent_items = apriori(my_basket_sets, min_support = 0.01,use_colnames = True)
frequent_items

,support,itemsets
0,0.036093,(Alfajores)
1,0.015948,(Baguette)
2,0.324940,(Bread)
3,0.039765,(Brownie)
4,0.103137,(Cake)
...,...,...
58,0.049523,"(Tea, Coffee)"
59,0.023502,"(Toast, Coffee)"
60,0.018256,"(Tea, NONE)"
61,0.014269,"(Sandwich, Tea)"


In [46]:
# now making the rules from frequent itemset generated above

rules = association_rules(frequent_items, metric = "lift", min_threshold = 1)
rules.sort_values('confidence', ascending = False, inplace = True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
32,(Toast),(Coffee),0.033365,0.475081,0.023502,0.704403,1.482699,0.007651,1.775789
30,(Spanish Brunch),(Coffee),0.018046,0.475081,0.010807,0.598837,1.260494,0.002233,1.308493
18,(Medialuna),(Coffee),0.061379,0.475081,0.034939,0.569231,1.198175,0.005779,1.218561
24,(Pastry),(Coffee),0.085510,0.475081,0.047214,0.552147,1.162216,0.006590,1.172079
0,(Alfajores),(Coffee),0.036093,0.475081,0.019515,0.540698,1.138116,0.002368,1.142861
16,(Juice),(Coffee),0.038296,0.475081,0.020460,0.534247,1.124537,0.002266,1.127031
23,(NONE),(Coffee),0.079005,0.475081,0.042073,0.532537,1.120938,0.004539,1.122908
26,(Sandwich),(Coffee),0.071346,0.475081,0.037981,0.532353,1.120551,0.004086,1.122468
6,(Cake),(Coffee),0.103137,0.475081,0.054349,0.526958,1.109196,0.005350,1.109667
28,(Scone),(Coffee),0.034309,0.475081,0.017941,0.522936,1.100729,0.001642,1.100310


In [47]:
# arranging the data from highest to lowest with respect to 'confidence'

rules.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
32,(Toast),(Coffee),0.033365,0.475081,0.023502,0.704403,1.482699,0.007651,1.775789
30,(Spanish Brunch),(Coffee),0.018046,0.475081,0.010807,0.598837,1.260494,0.002233,1.308493
18,(Medialuna),(Coffee),0.061379,0.475081,0.034939,0.569231,1.198175,0.005779,1.218561
24,(Pastry),(Coffee),0.085510,0.475081,0.047214,0.552147,1.162216,0.006590,1.172079
0,(Alfajores),(Coffee),0.036093,0.475081,0.019515,0.540698,1.138116,0.002368,1.142861
16,(Juice),(Coffee),0.038296,0.475081,0.020460,0.534247,1.124537,0.002266,1.127031
23,(NONE),(Coffee),0.079005,0.475081,0.042073,0.532537,1.120938,0.004539,1.122908
26,(Sandwich),(Coffee),0.071346,0.475081,0.037981,0.532353,1.120551,0.004086,1.122468
6,(Cake),(Coffee),0.103137,0.475081,0.054349,0.526958,1.109196,0.005350,1.109667
28,(Scone),(Coffee),0.034309,0.475081,0.017941,0.522936,1.100729,0.001642,1.100310


https://www.kaggle.com/code/aboliveira/bakery-market-basket-analysis


https://www.kaggle.com/code/nandinibagga/apriori-algorithm
    


**leerlo y entender el resultado**


Very cool! We clearly see meaningful results here from our analysis shown above, where the higher the lift value, the stronger the correlation between the items. The data clearly shows that coffee is a popular consequent, which makes sense because it is a bakery. Besides coffee, let's look at the more interesting item correlations (format: antecedant(s) -> consequent):

Pastry -> Bread
Cake -> Tea
(Coffee + Tea) -> Cake
Sandwhich -> Tea
Hot Chocolate -> Cake
So how is this useful knowledge for the bakery? Businesses are always looking to optimize their setup and drive up their sales. Bakeries are no different, and this kind of analysis could have been done for any kind of retail store or market place as well. Because we now know the correlation between items and the common interest of the customers, the business can make decisions based on these findings. For example, this bakery might want to place their freshly baked bread near their pastries, since customers who purchase pastries seem to also be enticed by bread. Besides product placement, the bakery might also be interested in having a promotion of a free item, given the great chances of another item being sold as a result of it (For example, if they were to give away some of their free special toast one day, it might not only attract new frequent customers, but there is also a very good chance that the customer will still spend money on coffee).

# FP Growth Algorithms